In [8]:
from const import TOKEN
import telebot
from telebot import types
import sqlite3 as sq
import datetime
import re


bot = telebot.TeleBot(TOKEN, parse_mode=None)

"""with open("BotT.txt", "r") as file:
    TOKEN = file.read().strip()

bot = telebot.TeleBot(TOKEN, parse_mode=None)"""

# create an empty dictionary to store user data
user_data = {}

# create date: 05.03.2023 and time: 12:00
date_1 = datetime.date(2023, 3, 5)
time_1 = datetime.time(12,0)

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"#.replace('-', '_')
#SQL doesn`t allow '-', so i have to replace it

# open file with database
with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
  cursor = connect.cursor()

#
if cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';").fetchone():
  cursor.execute(f"SELECT SUM(quantity) FROM {table_name}")
  total_quantity = cursor.fetchone()[0] # return tuple with 'Sum'
  if total_quantity is not None and total_quantity > 50:
  # create new table with group for the next hour
    datetime_1 = datetime.datetime.combine(date_1, time_1) + datetime.timedelta(hours=1) #combine to convert in 'date' object 
    time_1 = datetime_1.time() #cause adding hours is forbidden
  

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"

cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
            name STRING,
            quantity INTEGER,
            contact STRING UNIQUE
            );   """)
connect.commit()

@bot.message_handler(commands=['omet','start']) # comands that responded
#@bot.message_handler(content_types=['text','sticker'])
def respond(message):
  
# photo_mus = types.InputMediaPhoto('https://scontent-ams2-1.xx.fbcdn.net/v/t1.6435-9/193684250_1184653448716244_1549048136683557681_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=7yIjZg512oIAX-92GYU&_nc_ht=scontent-ams2-1.xx&oh=00_AfAudupE6CtYIyfwv4wymPOXc7HrylZBNvlL-KIs7B_Q3Q&oe=643A8642')
    
  text_menu=f"<b>{message.from_user.first_name}</b>.,виберіть та натисніть одну з кнопок нижче\n\
           1.Зареєструватись на найближчу екскурсію.\n\
           2.Відвідати наш сайт.\n\
           3.Залишити відгук відносно роботи електротранспорту."
  markup=types.InlineKeyboardMarkup(row_width=1) #, resize_keyboard=True)
    
      
  button_1=types.InlineKeyboardButton(f"1.Екскурсія\
    {date_1.strftime ('%d-%m-%Y')} о {time_1}",callback_data='schedule')
     #photo = photo.read(),\
      #photo = photo_mus)
  with open('photo_2023-03-13_10-20-36.jpg', 'rb') as photo:
    bot.send_photo(message.chat.id, photo, reply_markup=markup)
    
  button_2=types.InlineKeyboardButton("2.Відвідати офіційну сторінку КП 'OMET'",\
    url="https://oget.od.ua",callback_data='site')
  button_3=types.InlineKeyboardButton("3.Залишити відгук.", callback_data='respond')
  markup.add(button_1,button_2,button_3)

  bot.send_message(message.chat.id, text_menu, reply_markup=markup, parse_mode='html') #,resize_keybord=True)


def respond_buttons(message):
    markup = types.InlineKeyboardMarkup(row_width=3)

    text_button = types.InlineKeyboardButton("Текстовий відгук", callback_data='text_respond')
    audio_button = types.InlineKeyboardButton("Аудіо відгук", callback_data='audio_respond')
    video_button = types.InlineKeyboardButton("Відео відгук", callback_data='video_respond')

    markup.add(text_button, audio_button, video_button)
    
    bot.send_message(message.chat.id, "Оберіть форму вашого звернення:", reply_markup=markup)

@bot.callback_query_handler(func=lambda call:True)
def callback(call):
  if call.message: # check if button pushed
    if call.data == 'schedule':
      bot.register_next_step_handler(call.message, save_quantity)
      bot.send_message(call.message.chat.id, "Напишіть цифрами (від 1 до 50) кількість людей,\
              які планують відвідати музей КП 'ОМЕТ'\n\
              (наприклад:'12' - бот ідентифікує лише цифрову складову у цьому повідомленні)")
      
    if call.data == 'respond':
      respond_buttons(call.message)
    if call.data == 'text_respond':
          bot.register_next_step_handler(call.message, save_text_respond)
          bot.send_message(call.message.chat.id, "Введіть текстовий відгук\n\
            (нагадуємо, що в повідомленні телеграма може вміститись близько 4000 символів)")
    if call.data == 'audio_respond':
          bot.register_next_step_handler(call.message, save_audio_respond)
          bot.send_message(call.message.chat.id, "Надішліть аудіо відгук\n\
            тривалістю до 1 хвилини")
    if call.data == 'video_respond':
          bot.register_next_step_handler(call.message, save_video_respond)
          bot.send_message(call.message.chat.id, "Надішліть відео відгук\n\
            тривалістю до 1 хвилини")

def save_text_respond(message):
    # Збереження текстового відгука
    text_from_user = message

"""Номер інформаційного центру 717 54 54 або 788 688 6"""

# How to make a function that will send telegram message via mail?


def save_audio_respond(message):
    # Збереження аудіо відгука
    pass

def save_video_respond(message):
    # Збереження відео відгука
    pass


def give_respond(message):
  pass

def save_quantity(message):
  quantity = message.text.strip()  # remove extra spaces from text 
  
  user_data['name'] = message.from_user.first_name
    
  # check if quantity is real number
  if not quantity.isdigit():
    bot.send_message(message.chat.id, "Невірно введена кількість")
    bot.register_next_step_handler(message, save_quantity)  # waiting for next message
    return
  
  # check if quantity more than 50
  if int(quantity) > 50:
    bot.send_message(message.chat.id, "Зв'яжіться із директором музею за телефоном 050 399 42 11 для обговорення окремої\
      екскурсії для такої великої кількості людей.\n\
      Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'.")
    return
  
  # separate the numerical part from the text, if there is one
  digits_str = ''.join(re.findall('\d+', quantity))
  digits_int = int(digits_str) if digits_str else None
  
  user_data['quantity'] = digits_int
  
  bot.send_message(message.chat.id, f"Кількість відвідувачів {digits_int} збережено")
  
  bot.send_message(message.chat.id, "Надішліть будь ласка ваш номер телефону для зв'язку з вами,\n\
    наприклад: 0661234567 або 7562598 , без пробілів та інших символів крім цифр")
  
  bot.register_next_step_handler(message, ask_phone)
                   


def ask_phone(message):
  global table_name
  phone_number = message.text.strip()  # remove extra spaces from text
  int_phone = int(''.join(re.findall('\d+', phone_number)))
  
  # check if phone number is valid
  #if not phone_number.isdigit() or not 6 <= len(phone_number) <= 13:
  if not re.match(r'^\+?\d{6,13}$', phone_number):
    # '^' - begin of str; '\+?' - '+' could be included or not; '\d' - indicate any number, '{6,13} - quantity'; '$' - end of a str.
    bot.send_message(message.chat.id, "В номері невідповідна кількість цифр або він містить недопустимі символи")
    bot.register_next_step_handler(message, ask_phone)  # waiting for next message
    return
  
  with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
    cursor = connect.cursor()
  
  # check if phone number and name is unique
    cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE contact =? Or name =?", (int_phone, user_data['name']))
    count = cursor.fetchone()[0]
    if count > 0:
      bot.send_message(message.chat.id, "Номер телефону або користувача вже зареєстрований на цю екскурсію")
      bot.register_next_step_handler(message, ask_phone)  # waiting for next message
      return
  
  user_data['phone'] = int_phone
  
  cursor.execute(f"Insert Or Replace Into {table_name} (name, quantity, contact) VALUES (?, ?, ?)",\
    (user_data['name'], user_data['quantity'], user_data['phone']))
  connect.commit()
  
  connect.close()

  bot.send_message(message.chat.id, f"Ваш контакт збережено. Будемо раді бачити вас у музеї, гарного дня!\
    Якщо хочете повернутись в головне меню натисніть-напишіть команду '/start' або '/omet'")
  
  #bot.register_next_step_handler(message, respond)

  bot.register_next_step_handler(message, respond)

bot.infinity_polling()

2023-03-23 10:45:15,652 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-03-23 10:45:15,653 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email import encoders

def send_email(subject, text, img_path=None, audio_path=None, video_path=None):
    from_email = 'your_email@example.com'
    to_email = 'recipient@example.com'
    password = 'your_email_password'

    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(text, 'plain'))

    if img_path:
        with open(img_path, 'rb') as f:
            img_data = f.read()
            image = MIMEImage(img_data)
            msg.attach(image)

    if audio_path:
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
            audio = MIMEAudio(audio_data)
            msg.attach(audio)

    if video_path:
        with open(video_path, 'rb') as f:
            video_data = f.read()
            video = MIMEBase('application', 'octet-stream')
            video.set_payload(video_data)
            encoders.encode_base64(video)
            video.add_header('Content-Disposition', f'attachment; filename="{video_path}"')
            msg.attach(video)

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(from_email, password)
    server.sendmail(from_email, to_email, msg.as_string())
    server.quit()

send_email('Subject', 'Message text', img_path='path/to/image.jpg', audio_path='path/to/audio.mp3', video_path='path/to/video.mp4')


In [ ]:
# Команда /start
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привіт! Я бот для збереження текстових, фото, аудіо та відеовідгуків. Будь ласка, надішліть свій відгук.")

# Обробник текстових повідомлень
@bot.message_handler(content_types=['text'])
def save_text(message):
    # Зберігайте текстові відгуки тут
    bot.reply_to(message, "Текстовий відгук збережено!")

# Обробник фото
@bot.message_handler(content_types=['photo'])
def save_photo(message):
    # Зберігайте фото тут
    bot.reply_to(message, "Фото збережено!")

# Обробник аудіо
@bot.message_handler(content_types=['audio', 'voice'])
def save_audio(message):
    # Зберігайте аудіо тут
    bot.reply_to(message, "Аудіо збережено!")

# Обробник відео
@bot.message_handler(content_types=['video'])
def save_video(message):
    # Зберігайте відео тут
    bot.reply_to(message, "Відео збережено!")

# Запуск бота
if __name__ == '__main__':
    bot.polling(none_stop=True)

In [2]:
from const import TOKEN
import telebot
from telebot import types
import sqlite3 as sq
import datetime
import re


bot = telebot.TeleBot(TOKEN, parse_mode=None)

"""with open("BotT.txt", "r") as file:
    TOKEN = file.read().strip()

bot = telebot.TeleBot(TOKEN, parse_mode=None)"""

# create an empty dictionary to store user data
user_data = {}

# create date: 05.03.2023 and time: 12:00
date_1 = datetime.date(2023, 3, 5)
time_1 = datetime.time(12,0)

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"#.replace('-', '_')
#SQL doesn`t allow '-', so i have to replace it

# open file with database
with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
  cursor = connect.cursor()

#
if cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';").fetchone():
  cursor.execute(f"SELECT SUM(quantity) FROM {table_name}")
  total_quantity = cursor.fetchone()[0] # return tuple with 'Sum'
  if total_quantity is not None and total_quantity > 50:
  # create new table with group for the next hour
    datetime_1 = datetime.datetime.combine(date_1, time_1) + datetime.timedelta(hours=1) #combine to convert in 'date' object 
    time_1 = datetime_1.time() #cause adding hours is forbidden
  

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"

cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
            name STRING,
            quantity INTEGER,
            contact STRING UNIQUE
            );   """)
connect.commit()

@bot.message_handler(commands=['omet','start']) # comands that responded
#@bot.message_handler(content_types=['text','sticker'])
def respond(message):
  
# photo_mus = types.InputMediaPhoto('https://scontent-ams2-1.xx.fbcdn.net/v/t1.6435-9/193684250_1184653448716244_1549048136683557681_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=7yIjZg512oIAX-92GYU&_nc_ht=scontent-ams2-1.xx&oh=00_AfAudupE6CtYIyfwv4wymPOXc7HrylZBNvlL-KIs7B_Q3Q&oe=643A8642')
    
  text_menu=f"<b>{message.from_user.first_name}</b>.,виберіть та натисніть одну з кнопок нижче\n\
           1.Зареєструватись на найближчу екскурсію.\n\
           2.Відвідати наш сайт."
           #3.Залишити відгук відносно роботи електротранспорту."
  markup=types.InlineKeyboardMarkup(row_width=1) #, resize_keyboard=True)
    
      
  button_1=types.InlineKeyboardButton(f"1.Екскурсія\
    {date_1.strftime ('%d-%m-%Y')} о {time_1}",callback_data='schedule')
     #photo = photo.read(),\
      #photo = photo_mus)
  with open('photo_2023-03-13_10-20-36.jpg', 'rb') as photo:
    bot.send_photo(message.chat.id, photo, reply_markup=markup)
    
  button_2=types.InlineKeyboardButton("2.Відвідати офіційну сторінку КП 'OMET'",\
    url="https://oget.od.ua",callback_data='site')
  #button_3=types.InlineKeyboardButton("3.Залишити відгук.", callback_data='respond')
  markup.add(button_1,button_2)#,button_3)

  bot.send_message(message.chat.id, text_menu, reply_markup=markup, parse_mode='html') #,resize_keybord=True)


def respond_buttons(message):
    markup = types.InlineKeyboardMarkup(row_width=3)

    text_button = types.InlineKeyboardButton("Текстовий відгук", callback_data='text_respond')
    audio_button = types.InlineKeyboardButton("Аудіо відгук", callback_data='audio_respond')
    video_button = types.InlineKeyboardButton("Відео відгук", callback_data='video_respond')

    markup.add(text_button, audio_button, video_button)
    
    bot.send_message(message.chat.id, "Оберіть форму вашого звернення:", reply_markup=markup)

@bot.callback_query_handler(func=lambda call:True)
def callback(call):
  if call.message: # check if button pushed
    if call.data == 'schedule':
      bot.register_next_step_handler(call.message, save_quantity)
      bot.send_message(call.message.chat.id, "Напишіть цифрами (від 1 до 50) кількість людей,\
              які планують відвідати музей КП 'ОМЕТ'\n\
              (наприклад:'12' - бот ідентифікує лише цифрову складову у цьому повідомленні)")
      
    if call.data == 'respond':
      respond_buttons(call.message)
    if call.data == 'text_respond':
          bot.register_next_step_handler(call.message, save_text_respond)
          bot.send_message(call.message.chat.id, "Введіть текстовий відгук\n\
            (нагадуємо, що в повідомленні телеграма може вміститись близько 4000 символів)")
    if call.data == 'audio_respond':
          bot.register_next_step_handler(call.message, save_audio_respond)
          bot.send_message(call.message.chat.id, "Надішліть аудіо відгук\n\
            тривалістю до 1 хвилини")
    if call.data == 'video_respond':
          bot.register_next_step_handler(call.message, save_video_respond)
          bot.send_message(call.message.chat.id, "Надішліть відео відгук\n\
            тривалістю до 1 хвилини")

def save_text_respond(message):
    # Збереження текстового відгука
    text_from_user = message

"""Номер інформаційного центру 717 54 54 або 788 688 6"""

# How to make a function that will send telegram message via mail?


def save_audio_respond(message):
    # Збереження аудіо відгука
    pass

def save_video_respond(message):
    # Збереження відео відгука
    pass


def give_respond(message):
  pass

def save_quantity(message):
  quantity = message.text.strip()  # remove extra spaces from text 
  
  user_data['name'] = message.from_user.first_name
    
  # check if quantity is real number
  if not quantity.isdigit():
    bot.send_message(message.chat.id, "Невірно введена кількість")
    bot.register_next_step_handler(message, save_quantity)  # waiting for next message
    return
  
  # check if quantity more than 50
  if int(quantity) > 50:
    bot.send_message(message.chat.id, "Зв'яжіться із директором музею за телефоном 050 399 42 11 для обговорення окремої\
      екскурсії для такої великої кількості людей.\n\
      Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'.")
    return
  
  # separate the numerical part from the text, if there is one
  digits_str = ''.join(re.findall('\d+', quantity))
  digits_int = int(digits_str) if digits_str else None
  
  user_data['quantity'] = digits_int
  
  bot.send_message(message.chat.id, f"Кількість відвідувачів {digits_int} збережено")
  
  bot.send_message(message.chat.id, "Надішліть будь ласка ваш номер телефону для зв'язку з вами,\n\
    наприклад: 0661234567 або 7562598 , без пробілів та інших символів крім цифр")
  
  bot.register_next_step_handler(message, ask_phone)
                   


def ask_phone(message):
  global table_name
  phone_number = message.text.strip()  # remove extra spaces from text
  int_phone = int(''.join(re.findall('\d+', phone_number)))
  
  # check if phone number is valid
  #if not phone_number.isdigit() or not 6 <= len(phone_number) <= 13:
  if not re.match(r'^\+?\d{6,13}$', phone_number):
    # '^' - begin of str; '\+?' - '+' could be included or not; '\d' - indicate any number, '{6,13} - quantity'; '$' - end of a str.
    bot.send_message(message.chat.id, "В номері невідповідна кількість цифр або він містить недопустимі символи")
    bot.register_next_step_handler(message, ask_phone)  # waiting for next message
    return
  
  with sq.connect('mus_schedule.db', check_same_thread=False) as connect:
    cursor = connect.cursor()
  
  # check if phone number and name is unique
    cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE contact =? Or name =?", (int_phone, user_data['name']))
    count = cursor.fetchone()[0]
    if count > 0:
      bot.send_message(message.chat.id, "Номер телефону або користувача вже зареєстрований на цю екскурсію")
      bot.register_next_step_handler(message, ask_phone)  # waiting for next message
      return
  
  user_data['phone'] = int_phone
  
  cursor.execute(f"Insert Or Replace Into {table_name} (name, quantity, contact) VALUES (?, ?, ?)",\
    (user_data['name'], user_data['quantity'], user_data['phone']))
  connect.commit()
  
  connect.close()

  bot.send_message(message.chat.id, f"Ваш контакт збережено. Будемо раді бачити вас у музеї, гарного дня!\
    Якщо хочете повернутись в головне меню натисніть-напишіть команду '/start' або '/omet'")
  
  #bot.register_next_step_handler(message, respond)

  bot.register_next_step_handler(message, respond)

bot.infinity_polling()

2023-03-24 15:30:38,820 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-03-24 15:30:38,822 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [ ]:
"""помилка зазвичай виникає тоді, коли ви використовуєте одне з'єднання з базою даних з різних потоків. Щоб уникнути
цієї помилки, вам потрібно зберігати окреме з'єднання з базою даних для кожного потоку.
Одним з способів цього досягнути є використання менеджера контексту with. Ось як можна модифікувати код для
використання менеджера контексту:"""

import sqlite3
import telebot
from threading import Thread
from const import TOKEN

bot = telebot.TeleBot(TOKEN, parse_mode=None)

# Функція для зберігання повідомлень в базі даних
def save_message(message):
    # З'єднання з базою даних
    with sqlite3.connect('messages.db') as conn:
        cursor = conn.cursor()

        # Створення таблиці повідомлень
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS messages
            (id INTEGER PRIMARY KEY, chat_id INTEGER, message TEXT)
        ''')

        # Збереження повідомлення в базі даних
        chat_id = message.chat.id
        message_text = message.text
        cursor.execute('INSERT INTO messages(chat_id, message) VALUES(?, ?)', (chat_id, message_text))
        conn.commit()

# Обробник повідомлень
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    thread = Thread(target=save_message, args=(message,))
    thread.start()

# Запуск бота
bot.polling()

"""У цьому варіанті коду save_message зберігає повідомлення в базі даних з використанням окремого з'єднання з базою
даних для кожного потоку. Код handle_message створює новий потік для збереження повідомлення в базі даних, щоб не
блокувати основний потік виконання бота."""

In [ ]:
Для збереження фото, відео та аудіо повідомлень від користувачів бота можна використовувати метод getFile() та бібліотеку urllib для отримання фото, відео та аудіо файлів з Telegram серверів. Далі ці файли можна зберегти на локальному комп'ютері, наприклад, в папці "media".

Нижче наведений код для отримання фото, аудіо та відео повідомлень і збереження їх на локальному комп'ютері:

python
Copy code
import urllib
import os
import telebot

TOKEN = "YOUR_TOKEN"
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(content_types=['photo', 'audio', 'video'])
def save_files(message):
    # отримуємо інформацію про файл
    file_info = bot.get_file(message.document.file_id)
    file_path = file_info.file_path
    file_url = f"https://api.telegram.org/file/bot{TOKEN}/{file_path}"
    file_ext = os.path.splitext(file_path)[-1]
    
    # створюємо папку "media" якщо її не існує
    if not os.path.exists("media"):
        os.makedirs("media")
    
    # зберігаємо файл в папці "media" з назвою message_id і розширенням відповідного файлу
    filename = f"{message.message_id}{file_ext}"
    filepath = os.path.join("media", filename)
    urllib.request.urlretrieve(file_url, filepath)

    # відправляємо відповідь
    bot.reply_to(message, "Файл збережено")

bot.polling()